<a href="https://colab.research.google.com/github/marcelounb/Deep_Learning_with_python_JasonBrownlee/blob/master/10_1_Iris_Flowers_Classi%EF%AC%81cation_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Multiclass Classiﬁcation Of Flower Species

In this tutorial we will use the standard machine learning problem called the iris ﬂowers dataset. This dataset is well studied and is a good problem for practicing on neural networks because all of the 4 input variables are numeric and have the same scale in centimeters. Each instance describes the properties of an observed ﬂower measurements and the output variable is speciﬁc iris species. The attributes for this dataset can be summarized as follows:
1. Sepal length in centimeters.
2. Sepal width in centimeters.
3. Petal length in centimeters.
4. Petal width in centimeters.
5. Class (Iris-setosa, Iris-versicolour, Iris-virginica).


This is a multiclass classiﬁcation problem, meaning that there are more than two classes to be predicted, in fact there are three ﬂower species. This is an important type of problem on which to practice with neural networks because the three class values require specialized handling. Below is a sample of the ﬁrst ﬁve of the 150 instances:

5.1,3.5,1.4,0.2,Iris-setosa 

4.9,3.0,1.4,0.2,Iris-setosa 

4.7,3.2,1.3,0.2,Iris-setosa 

4.6,3.1,1.5,0.2,Iris-setosa 

5.0,3.6,1.4,0.2,Iris-setosa

The iris ﬂower dataset is a well studied problem and a such we can expect to achieve a model accuracy in the range of 95% to 97%. This provides a good target to aim for when developing our models.

In [0]:
import numpy as np
import pandas as pd 
from keras.models import Sequential 
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline

In [0]:
# load dataset 
dataframe = pd.read_csv("/content/iris.data", header=None) 
dataset = dataframe.values 
X = dataset[:,0:4].astype(float) 
Y = dataset[:,4]

Understanding the dataset

In [5]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [6]:
dataset[:10]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa']], dtype=object)

# Initialize Random Number Generator
Next we need to initialize the random number generator to a constant value. This is important to ensure that the results we achieve from this model can be achieved again precisely. It ensures that the stochastic process of training a neural network model can be reproduced.

In [0]:
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)

# Encode The Output Variable
The output variable contains three di↵erent string values. When modeling multiclass classiﬁcation problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not. This is called one hot encoding or creating dummy variables from a categorical variable. For example, in this problem the three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the three observations:
1. Iris-setosa 
2. Iris-versicolor 
3. Iris-virginic

We can turn this into a one-hot encoded binary matrix for each data instance that would look as follows:

Iris-setosa, Iris-versicolor, Iris-virginica 

1, 0, 0

0, 1, 0
 
0, 0, 1

We can do this by ﬁrst encoding the strings consistently to integers using the scikit-learn class LabelEncoder. Then convert the vector of integers to a one hot encoding using the Keras function to categorical()

In [0]:
# encode class values as integers 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y) 
# convert integers to dummy variables (i.e. one hot encoded) 
dummy_y = np_utils.to_categorical(encoded_Y)

In [9]:
encoded_Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [10]:
dummy_y[45:55]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Deﬁne The Neural Network Model
a simple fully connected network with one hidden layer that contains 4 neurons, the same number of inputs (it could be any number of neurons). 

The hidden layer uses a rectiﬁer activation function which is a good practice. Because we used a one-hot encoding for our iris dataset, the output layer must create 3 output values, one for each class. 

The output value with the largest value will be taken as the class predicted by the model. The network topology of this simple one-layer neural network can be summarized as:

4 inputs -> [4 hidden nodes] -> 3 outputs

Note that we use a sigmoid activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities. Finally, the network uses the ecient ADAM gradient descent optimization algorithm with a logarithmic loss function, which is called categorical crossentropy in Keras.



In [0]:
# define baseline model 
def baseline_model(): 
  # create model 
  model = Sequential() 
  model.add(Dense(4, input_dim=4, kernel_initializer= 'normal' , activation= 'relu' )) 
  model.add(Dense(3, kernel_initializer=  'normal' , activation= 'sigmoid' )) 
  # Compile model 
  model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

We can now create our KerasClassifier for use in scikit-learn. We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs 'epochs' as 200 and batch size as 5 to use when training the model. Debugging is also turned o↵ when training by setting verbose to 0.

In [0]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5)

# Evaluate The Model with k-Fold Cross Validation
  We can now evaluate the neural network model on our training data. The scikit-learn library has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation. 

First we can deﬁne the model evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.

In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

Now we can evaluate our model (estimator) on our dataset (X and dummy y) using a 10-fold cross validation procedure (kfold). Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset.


In [14]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold) 
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
135/135 [==============================] - 0s 2ms/step - loss: 1.0998 - accuracy: 0.2370
Epoch 2/200
135/135 [==============================] - 0s 326us/step - loss: 1.0963 - accuracy: 0.3481
Epoch 3/200
135/135 [==============================] - 0s 289us/step - loss: 1.0931 - accuracy: 0.3481
Epoch 4/200
135/135 [==============================] - 0s 242us/step - loss: 1.0882 - accuracy: 0.3481
Epoch 5/200
135/135 [==============================] - 0s 226us/step - loss: 1.0813 - accuracy: 0.3481
Epoch 6/200
135/135 [==============================] - 0s 245us/step - loss: 1.0706 - accuracy: 0.3481
Epoch 7/200
135/135 [==============================] - 0s 320us/step - loss: 1.0561 - accuracy: 0.3852
Epoch 8/200
135/135 [==============================] - 0s 264us/step - loss: 1.0374 - accuracy: 0.6000
Epoch 9/200
135/135 [==============================] - 0s 249us/step - loss: 1.0156 - accuracy: 0.6741
Epoch 10/200
135/135 [==============================] - 0s 238us/step - los